
# تمرین داده‌های آیریس – انتخاب بهترین فیچر، رسم پراکندگی و ماتریس درهم‌ریختگی

این نوت‌بوک مراحل زیر را انجام می‌دهد:
1) بارگذاری داده‌ی Iris  
2) ارزیابی **بهترین فیچر تکی** و **بهترین جفت فیچر** با دقت میانگین Cross-Validation  
3) رسم **نمودار پراکندگی** برای بهترین جفت فیچر  
4) آموزش یک مدل ساده (Decision Tree) و رسم **ماتریس درهم‌ریختگی** روی داده‌ی آزمون

> نکته: برای رسم نمودار از کتابخانه‌ی `matplotlib` استفاده شده و هیچ رنگ اختصاصی تعیین نشده است (مطابق محدودیت‌های تمرینی).


In [ ]:

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# 1) بارگذاری داده‌ها
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="species")
df = X.copy()
df["species"] = y
target_names = {i: n for i, n in enumerate(iris.target_names)}
df["species_name"] = df["species"].map(target_names)

print("ویژگی‌ها:", list(X.columns))
print("کلاس‌ها:", list(iris.target_names))
print("شکل داده:", X.shape)
df.head()



## ارزیابی فیچرها با Cross-Validation
برای هر فیچر تکی و هر جفت فیچر، یک **DecisionTreeClassifier** با 5-fold Stratified CV اجرا می‌شود و میانگین دقت محاسبه می‌شود.


In [ ]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = DecisionTreeClassifier(random_state=0)

# فیچرهای تکی
single_scores = []
for feat in X.columns:
    scores = cross_val_score(clf, X[[feat]], y, cv=cv)
    single_scores.append({"feature": feat, "cv_mean_acc": scores.mean(), "cv_std": scores.std()})
single_df = pd.DataFrame(single_scores).sort_values("cv_mean_acc", ascending=False).reset_index(drop=True)
print("Single feature CV results:")
display(single_df)

# جفت فیچرها
pair_scores = []
import itertools
for (f1, f2) in itertools.combinations(X.columns, 2):
    scores = cross_val_score(clf, X[[f1, f2]], y, cv=cv)
    pair_scores.append({"features": f"{f1} + {f2}", "f1": f1, "f2": f2, "cv_mean_acc": scores.mean(), "cv_std": scores.std()})
pair_df = pd.DataFrame(pair_scores).sort_values("cv_mean_acc", ascending=False).reset_index(drop=True)
print("Feature pair CV results:")
display(pair_df)

best_single = single_df.iloc[0]["feature"]
best_pair_f1 = pair_df.iloc[0]["f1"]
best_pair_f2 = pair_df.iloc[0]["f2"]

print("✔ بهترین فیچر تکی:", best_single)
print("✔ بهترین جفت فیچر:", best_pair_f1, "+", best_pair_f2)



## رسم نمودار پراکندگی برای بهترین جفت فیچر


In [ ]:

plt.figure(figsize=(6,5))
for k, name in target_names.items():
    idx = df["species"] == k
    plt.scatter(df.loc[idx, best_pair_f1], df.loc[idx, best_pair_f2], label=name)
plt.xlabel(best_pair_f1)
plt.ylabel(best_pair_f2)
plt.title("Iris – Best Feature Pair Scatter")
plt.legend()
plt.tight_layout()
plt.show()



## آموزش مدل و رسم ماتریس درهم‌ریختگی
مدل: **Decision Tree** روی بهترین جفت فیچر. داده‌ها به نسبت 70/30 به آموزش/آزمون تقسیم شده‌اند.


In [ ]:

X_pair = X[[best_pair_f1, best_pair_f2]]
X_train, X_test, y_train, y_test = train_test_split(X_pair, y, test_size=0.3, random_state=42, stratify=y)
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy on test: {acc:.3f}")

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(target_names.values()))
disp.plot(values_format='d', cmap=None)  # رنگ اختصاصی تعیین نشده
plt.title(f"Confusion Matrix (Acc={{acc:.3f}})")
plt.tight_layout()
plt.show()
